In [1]:
import lancedb
import pyarrow as pa
import pandas as pd
from datasets import load_dataset
from transformers import GPT2Tokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Para este ejercicio descargamos una base de datos de mensajes de texto en inglés.

In [ ]:
dataset = load_dataset("chirunder/text_messages")

In [ ]:
df = pd.DataFrame(dataset['train'])
df.rename(columns={'text': 'texto'}, inplace=True)
df.head()

Para los siguientes ejercicios, voy a crear una variable del numero de palabras en cada mensaje de texto.

In [ ]:
df['n'] = df['texto'].apply(lambda x: len(str(x).split()))
df = df[['n', 'texto']]
df.head(5000)

## Task 1:
A partir del dataframe df, crea df_tokenized (usando el Tokenizer de GPT2) con dos columnas pero con el texto tokenizado. Asegurate de que todos los embeddings tengan la misma longitud y los tokens sean enteros (todos enteros o todos doubles). 

In [ ]:

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


tokenized_text = df['texto'].apply(lambda x: tokenizer(x)["input_ids"])

tokenized_text = tokenized_text.apply(lambda x: x[:300] + [0] * (300 - len(x)) if len(x) < 300 else x[:300])


df_tokenized = pd.DataFrame({'vector': tokenized_text, 'name': df['n']})

df_tokenized.head()

## Task 2:
Mete el dataframe a una tabla en una base de datos de LanceDB.

In [ ]:
# Resuelve el task 2 aqui
db = lancedb.connect("./.lancedb")
# Creamos una tabla en la base de datos
db.create_table("tabla", df_tokenized)
db["tabla"].head()

## Task 3:
Haz una query estilo SQL a la tabla de la base de datos. Quiero que escribas la query equivalente y pongas la explicación de lo que está haciendo la consulta. Hint: usa la columna "n". 

In [ ]:
# Resuelve el task 3 aqui

In [ ]:
# Inicializar el tokenizador
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Definir las palabras clave relacionadas con sustancias ilícitas y tokenizarlas
keywords = ["weed", "alcohol", "drink", "smoke", "drug"]
tokenized_keywords = [tokenizer(keyword)["input_ids"][0] for keyword in keywords]

# Crear la consulta para buscar los textos que contienen alguna de las palabras clave tokenizadas
query = " OR ".join([f'vector LIKE "%{token}%"' for token in tokenized_keywords])

# Ejecutar la consulta en la base de datos
result_df = (
    db["tabla"]
    .search()
    .where(query)
    .select(["name", "vector"])
    .limit(11)
    .to_pandas()
)

#lo que estoy haciendo es selecionar todos los textos que tengan algo que ver con
#sustancias ilicitas para esto defini algunas keyword en las que creo que se centraria los mensajes
#pero no puedo meterle las palabras simplemente ya que el db ya está tokenizado

- Query en SQL equivalente:
- Explicacion: 

## Task 4:
Inventa un mensaje de texto que tu podrías escribirle a un amigo. Tokenizalo y ponlo en el formato adecuado para hacer un vector query. Quiero que me regreses el mensaje más parecido al mensaje que inventaste (OJO: quiero el texto, no el embedding). HINT: Hay que decodear el resultado del query.

In [ ]:
nuevo_mensaje="lets got to a party and drink a lot "
nuevo_mensaje_embeded = tokenizer(nuevo_mensaje)["input_ids"]

n = 300
def ajustar_vector(input, n):
    output = input[:n]
    
    # Si la lista es más corta que el tamaño objetivo, rellenar con 0.0
    while len(output) < n:
        output.append(0)
    
    return output
nuevo_mensaje_embeded = ajustar_vector(nuevo_mensaje_embeded, n)

In [ ]:
# Resuelve el task 4 aqui
result_df = (
    db["tabla"]
    .search(nuevo_mensaje_embeded)
    .metric("cosine")
    .select(["name", "vector"])
    .limit(1)
    .to_pandas()
)

mensaje_similar = result_df["name"].iloc[0]

# Obtener el texto original del mensaje más similar
texto_mensaje_similar = tokenizer.decode(df[df["n"] == mensaje_similar]["vector"].iloc[0])
